In [2]:
import numpy as np
import pandas as pd
import scipy
from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import train_test_split
import gc
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.metrics import mean_squared_log_error

In [3]:
df_train = pd.read_csv('/kaggle/input/dataset/train.tsv', sep='\t', low_memory=True)
print(df_train.shape)
print(df_train.info())
display(df_train.head())

(1482535, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1482535 entries, 0 to 1482534
Data columns (total 8 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   train_id           1482535 non-null  int64  
 1   name               1482535 non-null  object 
 2   item_condition_id  1482535 non-null  int64  
 3   category_name      1476208 non-null  object 
 4   brand_name         849853 non-null   object 
 5   price              1482535 non-null  float64
 6   shipping           1482535 non-null  int64  
 7   item_description   1482529 non-null  object 
dtypes: float64(1), int64(3), object(4)
memory usage: 90.5+ MB
None


,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity


In [4]:
df_test = pd.read_csv('/kaggle/input/dataset/test.tsv', sep='\t', low_memory=True)
print(df_test.shape)
print(df_test.info())
display(df_test.head())

(693359, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 693359 entries, 0 to 693358
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   test_id            693359 non-null  int64 
 1   name               693359 non-null  object
 2   item_condition_id  693359 non-null  int64 
 3   category_name      690301 non-null  object
 4   brand_name         397834 non-null  object
 5   shipping           693359 non-null  int64 
 6   item_description   693359 non-null  object
dtypes: int64(3), object(4)
memory usage: 37.0+ MB
None


,test_id,name,item_condition_id,category_name,brand_name,shipping,item_description
0,0,"Breast cancer ""I fight like a girl"" ring",1,Women/Jewelry/Rings,NaN,1,Size 7
1,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers",1,Other/Office supplies/Shipping Supplies,NaN,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers Lined..."
2,2,Coach bag,1,Vintage & Collectibles/Bags and Purses/Handbag,Coach,1,Brand new coach bag. Bought for [rm] at a Coac...
3,3,Floral Kimono,2,Women/Sweaters/Cardigan,NaN,0,-floral kimono -never worn -lightweight and pe...
4,4,Life after Death,3,Other/Books/Religion & Spirituality,NaN,1,Rediscovering life after the loss of a loved o...


In [5]:
%%time
def wordCount(text):
    """
    Parameters:
      text(str): 商品名、商品の説明文
    """
    try:
        if text == 'No description yet':
            return 0  # 商品名や説明が'No description yet'の場合は0を返す
        else:
            text = text.lower()                  # すべて小文字にする
            words = [w for w in text.split(" ")] # スペースで切り分ける
            return len(words)                    # 単語の数を返す
    except: 
        return 0

# 'name'の各フィールドの単語数を'name_len'に登録
df_train['name_len'] = df_train['name'].apply(lambda x: wordCount(x))
df_test['name_len'] = df_test['name'].apply(lambda x: wordCount(x))
# 'item_description'の各フィールドの単語数を'desc_len'に登録
df_train['desc_len'] = df_train['item_description'].apply(lambda x: wordCount(x))
df_test['desc_len'] = df_test['item_description'].apply(lambda x: wordCount(x))

CPU times: user 17.1 s, sys: 0 ns, total: 17.1 s
Wall time: 17.1 s


In [6]:
%%time
# 訓練データの'price'を対数変換する
df_train["target"] = np.log1p(df_train.price)

CPU times: user 35.6 ms, sys: 0 ns, total: 35.6 ms
Wall time: 33.1 ms


In [7]:
%%time
def split_cat(text):
    """
    Parameters:
      text(str): カテゴリ名

    ・カテゴリを/で切り分ける
    ・データが存在しない場合は"No Label"を返す
    """
    try: return text.split("/")
    except: return ("No Label", "No Label", "No Label")

# 3つに切り分けたカテゴリ名を'subcat_0'、'subcat_1'、'subcat_2'に登録
# 訓練データ
df_train['subcat_0'], df_train['subcat_1'], df_train['subcat_2'] = \
    zip(*df_train['category_name'].apply(lambda x: split_cat(x)))
# テストデータ
df_test['subcat_0'], df_test['subcat_1'], df_test['subcat_2'] = \
    zip(*df_test['category_name'].apply(lambda x: split_cat(x)))

CPU times: user 9.77 s, sys: 507 ms, total: 10.3 s
Wall time: 10.3 s


In [8]:
%%time
# df_trainとdf_testを縦方向に結合
full_set = pd.concat([df_train, df_test])
# full_setの'brand_name'から重複なしのブランドリスト(集合)を生成
all_brands = set(full_set['brand_name'].values)

# 'brand_name'の欠損値NaNを'missing'に置き換える
df_train['brand_name'].fillna(value='missing', inplace=True)
df_test['brand_name'].fillna(value='missing', inplace=True)

# 訓練データの'brand_name'が'missing'に一致するレコード数を取得
train_premissing = len(df_train.loc[df_train['brand_name'] == 'missing'])
# テストデータの'brand_name'が'missing'に一致するレコード数を取得
test_premissing = len(df_test.loc[df_test['brand_name'] == 'missing'])

def brandfinder(line):
    """
    Parameters: line(str): ブランド名

    ・ブランド名の'missing'を商品名に置き換える:
         missing'の商品名の単語がブランドリストに存在する場合
    ・ブランド名を商品名に置き換える:
        商品名がブランドリストの名前と完全に一致する場合
    ・ブランド名をそのままにする:
        商品名がブランドリストの名前と一致しない
        商品名が'missing'だが商品名の単語がブランドリストにない
    """
    brand = line[0] # 第1要素はブランド名
    name = line[1]  # 第2要素は商品名
    namesplit = name.split(' ') # 商品名をスペースで切り分ける
    
    if brand == 'missing':  # ブランド名が'missing'と一致
        for x in namesplit: # 商品名から切り分けた単語を取り出す
            if x in all_brands:                
                return name # 単語がブランドリストに一致したら商品名を返す
    if name in all_brands:  # 商品名がブランドリストに存在すれば商品名を返す
        return name
    
    return brand            # どれにも一致しなければブランド名を返す

# ブランド名の付替えを実施
df_train['brand_name'] = df_train[['brand_name','name']].apply(brandfinder, axis = 1)
df_test['brand_name'] = df_test[['brand_name','name']].apply(brandfinder, axis = 1)

# 書き換えられた'missing'の数を取得
train_found = train_premissing-len(df_train.loc[df_train['brand_name'] == 'missing'])
test_found = test_premissing-len(df_test.loc[df_test['brand_name'] == 'missing'])
print(train_premissing) # 書き換える前の'missing'の数
print(train_found)      # 書き換えられた'missing'の数
print(test_premissing)  # 書き換える前の'missing'の数
print(test_found)       # 書き換えられた'missing'の数

<timed exec>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


<timed exec>:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to 

632682
137418
295525
64154
CPU times: user 1min 46s, sys: 526 ms, total: 1min 47s
Wall time: 1min 47s


In [9]:
%%time
train_dfs, dev_dfs = train_test_split(
    df_train,         # 対象のデータフレーム
    random_state=123, # 乱数生成時のシード(種)
    train_size=0.99,  # 訓練用に99%のデータ
    test_size=0.01)   # 検証用に1%のデータ

n_trains = train_dfs.shape[0] # 訓練データのサイズ
n_devs = dev_dfs.shape[0]     # 検証データのサイズ
n_tests = df_test.shape[0]   # テストデータのサイズ
print('Training :', n_trains, 'examples')
print('Validating :', n_devs, 'examples')
print('Testing :', n_tests, 'examples')

del df_train
gc.collect()

Training : 1467709 examples
Validating : 14826 examples
Testing : 693359 examples
CPU times: user 1.59 s, sys: 64.1 ms, total: 1.66 s
Wall time: 1.65 s


0

In [10]:
%%time
# 訓練データ、検証データ、テストデータを1つのデータフレームに連結
df_all = pd.concat([train_dfs, dev_dfs, df_test])

print("Handling missing values...")
# カテゴリ名の欠損値を'missing'に置き換える
df_all['category_name'] = \
    df_all['category_name'].fillna('missing').astype(str)
# サブカテゴリのラベルを文字列に変換
df_all['subcat_0'] = df_all['subcat_0'].astype(str)
df_all['subcat_1'] = df_all['subcat_1'].astype(str)
df_all['subcat_2'] = df_all['subcat_2'].astype(str)
# ブランド名の欠損値を'missing'に置き換える
df_all['brand_name'] = df_all['brand_name'].fillna('missing').astype(str)
# 送料負担、商品の状態を文字列に置き換える
df_all['shipping'] = df_all['shipping'].astype(str)
df_all['item_condition_id'] = df_all['item_condition_id'].astype(str)
# 説明文の単語数、商品名の単語数を文字列に置き換える
df_all['desc_len'] = df_all['desc_len'].astype(str)
df_all['name_len'] = df_all['name_len'].astype(str)
# 説明文の欠損値を'No description yet'に置き換える
df_all['item_description'] = \
    df_all['item_description'].fillna('No description yet').astype(str)

Handling missing values...
CPU times: user 6.78 s, sys: 564 ms, total: 7.34 s
Wall time: 7.34 s


In [11]:
%%time
print("Vectorizing data...")
# CountVectorizerの生成処理を関数化
default_preprocessor = CountVectorizer().build_preprocessor()

def build_preprocessor(field):
    """
    指定されたカラムのインデックスを取得し、
    トークンカウント行列を作成するためのCountVectorizerを返す
    
    Parameter:field(str)
      フル結合データフレームのカラム名
    """
    field_idx = list(df_all.columns).index(field)
    return lambda x: default_preprocessor(x[field_idx])

# トークンカウント行列
# CountVectorizeを結合して
# (識別子, ベクトライザーオブジェクト)のリストで構成される
# トランスファーマーオブジェクトを生成
vectorizer = FeatureUnion([
    ('name', CountVectorizer(
        # bag-of-wordで分割する単位をn-gramで連続する単語のつながりとする
        # 商品名は2-gram(2つの単語のつながり)で分割
        ngram_range=(1, 2),
        max_features=1000, # トークンカウントの上限値############### 50000
        # トークンカウントステップをオーバーライド
        preprocessor=build_preprocessor('name'))),
    ('subcat_0', CountVectorizer(
        # トークンの構成を示す正規表現を'+'として1文字に対応
        token_pattern='.+',
        preprocessor=build_preprocessor('subcat_0'))),
    ('subcat_1', CountVectorizer(
        token_pattern='.+',
        preprocessor=build_preprocessor('subcat_1'))),
    ('subcat_2', CountVectorizer(
        token_pattern='.+',
        preprocessor=build_preprocessor('subcat_2'))),
    ('brand_name', CountVectorizer(
        token_pattern='.+',
        preprocessor=build_preprocessor('brand_name'))),
    ('shipping', CountVectorizer(
        token_pattern='\d+',
        preprocessor=build_preprocessor('shipping'))),
    ('item_condition_id', CountVectorizer(
        token_pattern='\d+',
        preprocessor=build_preprocessor('item_condition_id'))),
    ('desc_len', CountVectorizer(
        token_pattern='\d+',
        preprocessor=build_preprocessor('desc_len'))),
    ('name_len', CountVectorizer(
        token_pattern='\d+',
        preprocessor=build_preprocessor('name_len'))),
    ('item_description', TfidfVectorizer(
        # bag-of-wordで分割する単位をn-gramで連続する単語のつながりとする
        # 商品説名は3-gram(3つの単語のつながり)で分割
        ngram_range=(1, 3),
        max_features=1000, # トークンカウントの上限値#################### 100000
        preprocessor=build_preprocessor('item_description'))),
])

# フル結合のデータフレームのフィールド値を
# n-grainによるbag-of-wordsでトークンカウント行列に変換する
X = vectorizer.fit_transform(df_all.values)

del vectorizer
gc.collect()

# 入力用のdictオブジェクトから訓練用のデータを抽出
X_train = X[:n_trains]
# 訓練用のデータフレームから商品価格を抽出して
# (データ数, 価格)の2階テンソルに変換
Y_train = train_dfs.target.values.reshape(-1, 1)

# 入力用のdictオブジェクトから検証用のデータを抽出
X_dev = X[n_trains:n_trains+n_devs]
# 検証用のデータフレームから商品価格を抽出して
# (データ数, 価格)の2階テンソルに変換
Y_dev = dev_dfs.target.values.reshape(-1, 1)

# 入力用のdictオブジェクトからテストデータを抽出
X_test = X[n_trains+n_devs:]

print('X:', X.shape)
print('X_train:', X_train.shape)
print('X_dev:', X_dev.shape)
print('X_test:', X_test.shape)
print('Y_train:', Y_train.shape)
print('Y_dev:', Y_dev.shape)

Vectorizing data...
X: (2175894, 175913)
X_train: (1467709, 175913)
X_dev: (14826, 175913)
X_test: (693359, 175913)
Y_train: (1467709, 1)
Y_dev: (14826, 1)
CPU times: user 9min 23s, sys: 13.1 s, total: 9min 36s
Wall time: 9min 36s


In [15]:
%%time
print("Fitting Ridge model on training examples...")
ridge_model = Ridge(
    solver='auto',      # ソルバーをオートモードにする
    fit_intercept=True, # 切片を計算に使用
    alpha=1.0,          # 正則化の強度はデフォルト値
    max_iter=200,       # ソルバーの最大反復回数
    tol=0.01,           # 回帰の反復を停止するときの精度
    # データをシャッフルするときに使用する疑似乱数ジェネレータのシード
    random_state = 1,
)

print("Fitting RidgeCV model on training examples...")
ridge_modelCV = RidgeCV(
    fit_intercept=True, # 切片を計算に使用
    alphas=[5.0],
    cv = 2, # 交差検証時にスコアを2回連続して(毎回異なる分割で)計算
    # モデルの評価は平均2乗誤差回帰損失で行う
    scoring='neg_mean_squared_error',
)

ridge_model.fit(X_train, Y_train)
ridge_modelCV.fit(X_train, Y_train)

Fitting Ridge model on training examples...
Fitting RidgeCV model on training examples...
CPU times: user 12min 8s, sys: 9min 48s, total: 21min 56s
Wall time: 6min 29s


RidgeCV(alphas=[5.0], cv=2, scoring='neg_mean_squared_error')

In [19]:
Y_dev_preds_ridge = ridge_model.predict(X_dev)
Y_dev_preds_ridge = Y_dev_preds_ridge.reshape(-1, 1)
print('Ridge model RMSL error:', mean_squared_log_error(Y_dev, Y_dev_preds_ridge))

Ridge model RMSL error: 0.017227598532414803


In [17]:
# RidgeCVモデルに検証データを入力して損失を測定
Y_dev_preds_ridgeCV = ridge_modelCV.predict(X_dev)
Y_dev_preds_ridgeCV = Y_dev_preds_ridgeCV.reshape(-1, 1)
print('RidgeCV model RMSL error:', mean_squared_log_error(Y_dev, Y_dev_preds_ridgeCV))

RidgeCV model RMSL error: 0.01705041819933213


値が異常に低いので、RMSE関数を自作して計算してみる

In [20]:
# RMSE(Root Mean Square Error):二乗平均平方根誤差
# この関数を使用する際のYとY_predはすでにlogスケールになっているので、RMSLE(対数二乗平均平方根誤差)のように機能する
def rmsle(Y, Y_pred):
    assert Y.shape == Y_pred.shape
    return np.sqrt(np.mean(np.square(Y_pred - Y )))

In [21]:
Y_dev_preds_ridge = ridge_model.predict(X_dev)
Y_dev_preds_ridge = Y_dev_preds_ridge.reshape(-1, 1)
print('Ridge model RMSL error:', rmsle(Y_dev, Y_dev_preds_ridge))

Ridge model RMSL error: 0.5157330525857923


In [24]:
# RidgeCVモデルに検証データを入力して損失を測定
Y_dev_preds_ridgeCV = ridge_modelCV.predict(X_dev)
Y_dev_preds_ridgeCV = Y_dev_preds_ridgeCV.reshape(-1, 1)
print('RidgeCV model RMSL error:', rmsle(Y_dev, Y_dev_preds_ridgeCV))

RidgeCV model RMSL error: 0.5125338609304031
